In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

# _df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-11-19T00%3A00%3A00.000Z')
# df = df.append(_df, ignore_index=True)
# time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df

datafile = '../database/market/EOS_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

200

200

161

,date,open,high,low,close,volumn
0,2019-06-02T16:00:00.000Z,7.698000,7.7780,7.455000,7.5630,9.206138e+06
1,2019-06-01T16:00:00.000Z,8.189000,8.3970,7.318000,7.7000,3.179159e+07
2,2019-05-31T16:00:00.000Z,7.781000,8.7070,7.664000,8.1890,3.271501e+07
3,2019-05-30T16:00:00.000Z,8.327000,8.4990,6.800000,7.7810,4.446850e+07
4,2019-05-29T16:00:00.000Z,8.151000,8.3510,7.828000,8.3290,1.851198e+07
5,2019-05-28T16:00:00.000Z,7.962000,8.2300,7.520000,8.1510,2.210831e+07
6,2019-05-27T16:00:00.000Z,7.435000,8.3010,7.300000,7.9600,4.171901e+07
7,2019-05-26T16:00:00.000Z,6.321000,7.5200,6.283000,7.4290,3.057263e+07
8,2019-05-25T16:00:00.000Z,6.350000,6.4100,6.200000,6.3200,1.034164e+07
9,2019-05-24T16:00:00.000Z,6.389000,6.5950,6.266000,6.3480,1.842667e+07


In [5]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/BTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/EOS-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df

datafile = '../database/market/BITCOIN_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

200

200

200

0

,date,open,high,low,close,volumn
0,2019-06-02T16:00:00.000Z,8678.30,8760.00,8522.20,8580.00,66975.255083
1,2019-06-01T16:00:00.000Z,8528.70,8812.50,8467.60,8679.30,120180.417216
2,2019-05-31T16:00:00.000Z,8353.70,8623.60,8288.50,8528.80,121766.554858
3,2019-05-30T16:00:00.000Z,8929.90,9065.50,8000.00,8348.40,142904.675618
4,2019-05-29T16:00:00.000Z,8681.60,8942.10,8545.30,8928.70,117921.040518
5,2019-05-28T16:00:00.000Z,8686.00,8788.00,8420.00,8682.70,122317.985909
6,2019-05-27T16:00:00.000Z,8800.60,8903.70,8500.00,8686.40,120229.769070
7,2019-05-26T16:00:00.000Z,7982.10,8882.60,7936.30,8798.00,139212.961304
8,2019-05-25T16:00:00.000Z,8015.10,8061.20,7852.00,7982.10,110558.886078
9,2019-05-24T16:00:00.000Z,8049.10,8123.90,7900.00,8014.00,125061.662169


In [6]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/ETH-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df

datafile = '../database/market/ETHEREUM_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

200

200

200

1

,date,open,high,low,close,volumn
0,2019-06-02T16:00:00.000Z,267.79,270.60,264.02,265.98,4.608186e+05
1,2019-06-01T16:00:00.000Z,265.63,273.18,261.12,267.78,9.174507e+05
2,2019-05-31T16:00:00.000Z,257.60,276.45,254.33,265.68,1.212770e+06
3,2019-05-30T16:00:00.000Z,286.63,288.17,236.99,257.64,1.790621e+06
4,2019-05-29T16:00:00.000Z,271.76,288.00,266.65,286.65,1.089414e+06
5,2019-05-28T16:00:00.000Z,270.84,276.13,260.27,271.76,1.070878e+06
6,2019-05-27T16:00:00.000Z,269.23,279.95,262.75,270.77,1.301199e+06
7,2019-05-26T16:00:00.000Z,248.64,273.00,247.30,269.23,1.391023e+06
8,2019-05-25T16:00:00.000Z,250.88,252.06,245.20,248.55,8.615621e+05
9,2019-05-24T16:00:00.000Z,251.69,257.28,245.73,250.94,1.201943e+06


In [2]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/XRP-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df

datafile = '../database/market/RIPPLE_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

200

200

170

0

,date,open,high,low,close,volumn
0,2019-06-02T16:00:00.000Z,0.43810,0.46500,0.42850,0.44590,1.826774e+08
1,2019-06-01T16:00:00.000Z,0.42810,0.44570,0.42380,0.43810,1.153378e+08
2,2019-05-31T16:00:00.000Z,0.42080,0.44190,0.41790,0.42810,1.364100e+08
3,2019-05-30T16:00:00.000Z,0.47000,0.47300,0.39800,0.42070,2.577775e+08
4,2019-05-29T16:00:00.000Z,0.44810,0.47290,0.43520,0.47000,2.026603e+08
5,2019-05-28T16:00:00.000Z,0.44390,0.46000,0.42050,0.44810,2.630403e+08
6,2019-05-27T16:00:00.000Z,0.42280,0.46600,0.41130,0.44390,2.339971e+08
7,2019-05-26T16:00:00.000Z,0.38140,0.42650,0.37890,0.42270,2.116522e+08
8,2019-05-25T16:00:00.000Z,0.38450,0.38680,0.37390,0.38130,6.977798e+07
9,2019-05-24T16:00:00.000Z,0.38920,0.39310,0.37970,0.38440,1.076852e+08


In [9]:
df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2019-06-04T00%3A00%3A00.000Z')
len(df)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-11-15T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2018-04-29T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

_df = pd.read_json('https://www.okex.me/api/spot/v3/instruments/LTC-USDT/candles?granularity=86400&start=2017-01-01T00%3A00%3A00.000Z&end=2017-10-11T00%3A00%3A00.000Z')
len(_df)
df = df.append(_df, ignore_index=True)
time.sleep(1)

df.columns = ['date', 'open', 'high', 'low', 'close', 'volumn']
df

datafile = '../database/market/LITECOIN_OKEX.csv'
df.to_csv(datafile, header=['date', 'open', 'high', 'low', 'close', 'volume'], index=False, encoding='utf-8')

200

200

180

0

,date,open,high,low,close,volumn
0,2019-06-02T16:00:00.000Z,113.71,116.18,112.35,114.20,3.337653e+05
1,2019-06-01T16:00:00.000Z,112.75,116.50,111.10,113.71,7.520208e+05
2,2019-05-31T16:00:00.000Z,108.75,116.37,107.30,112.74,9.936538e+05
3,2019-05-30T16:00:00.000Z,119.08,120.29,102.00,108.75,1.595308e+06
4,2019-05-29T16:00:00.000Z,116.92,119.26,113.15,119.05,8.372367e+05
5,2019-05-28T16:00:00.000Z,114.03,118.56,108.66,116.94,9.676555e+05
6,2019-05-27T16:00:00.000Z,116.96,122.00,109.40,114.03,1.506858e+06
7,2019-05-26T16:00:00.000Z,100.50,117.80,99.70,116.95,1.693823e+06
8,2019-05-25T16:00:00.000Z,101.36,103.91,98.80,100.49,9.785822e+05
9,2019-05-24T16:00:00.000Z,99.39,105.00,98.18,101.35,1.644058e+06
